<a href="https://colab.research.google.com/github/code-with-ayesha/chatbots/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip -q install gradio langchain langchain_groq langchain_core langchain_community tavily-python

In [10]:
from google.colab import userdata
TAVELI_API_KEY = userdata.get('TAVELI_API_KEY')

In [11]:
import gradio as gr
from langchain_groq import ChatGroq
from google.colab import userdata


# Fetch your API key from userdata
groq_api_key = userdata.get('GROQ_API_KEY')

# Initialize the Groq model (e.g., "llama-3.1-70b-versatile")
llm = ChatGroq(model_name="llama-3.1-70b-versatile", temperature=1, api_key=groq_api_key)

# Function to check if the query is student-related
def is_student_query(query):
    # List of keywords related to student topics (you can extend this)
    student_keywords = ["homework", "study", "math", "science", "history", "assignment", "school", "exam","spelling","wiki"]
    return any(keyword.lower() in query.lower() for keyword in student_keywords)

# Function to handle student queries using Groq
def student_chatbot(student_input):
    try:
        # Check if the query is relevant to students
        if not is_student_query(student_input):
            return "This chatbot is only for educational questions. Please ask something related to your studies or school."

        # Modify the prompt to be more educational and relevant to the query
        prompt = f"You are a helpful assistant for students. Answer the following question in a way that helps a student understand the topic better:\n{student_input}"

        # Send the student's question to Groq's API and get a response
        response = llm.invoke(prompt)

        # Access the content directly from the response
        content = response.content  # Accessing content directly
        return content.strip()  # Return the content text

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Gradio Interface
interface = gr.Interface(
    fn=student_chatbot,  # This is the function to call when input is provided
    inputs=gr.Textbox(label="Ask Your Question", placeholder="Enter your question here..."),
    outputs=gr.Textbox(label="Answer"),
    title="Student Helper Chatbot",  # Title of the Gradio app
    description="Ask me any educational question, and I'll help you learn!"  # Student-focused description
)

# Launch the app
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b06c3e0e13c960a27f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [18]:
!pip -q install wikipedia

In [19]:
!pip -q install wikipedia-api


In [14]:
!pip install pyspellchecker

In [15]:
import os
os.environ["TAVELI_API_KEY"] = userdata.get('TAVELI_API_KEY')

In [16]:
import gradio as gr
from langchain_groq import ChatGroq
from google.colab import userdata
import wikipedia
import sympy as sp
from spellchecker import SpellChecker
import wikipedia


# Fetch your API keys from userdata
groq_api_key = userdata.get('GROQ_API_KEY')
taveli_api_key = userdata.get('TAVELI_API_KEY')

# Initialize the Groq model
llm = ChatGroq(model_name="llama-3.1-70b-versatile", temperature=1, api_key=groq_api_key)


# Initialize the spell checker
spell = SpellChecker()

# Tool 1: Spell Checker
def check_spelling(text):
    corrected_words = [spell.correction(word) for word in text.split()]
    return " ".join(corrected_words)

# Tool 2: Wikipedia Search

def search_wikipedia(query):
    try:
        # Search Wikipedia for a summary of the query
        summary = wikipedia.summary(query, sentences=3)
        return summary
    except wikipedia.exceptions.DisambiguationError as e:
        # Handle ambiguous queries
        options = ", ".join(e.options[:5])  # Suggest up to 5 options
        return f"The query '{query}' is ambiguous. Did you mean one of the following? {options}"
    except wikipedia.exceptions.PageError:
        # Handle queries with no matching page
        return f"No page found for '{query}'. Please try another search term."
    except Exception as e:
        # Handle any other exceptions
        return f"Could not find relevant information: {str(e)}"



# Tool 3: Calculator
import re


def calculate_expression(expression):
    try:
        # Use a regular expression to extract the mathematical expression
        math_expression = re.search(r"[\d\.\+\-\*/\^\(\)]+", expression)

        if math_expression:
            result = sp.sympify(math_expression.group())
            return str(result)  # Return the calculated result as a string
        else:
            return "No valid mathematical expression found in the input."
    except Exception as e:
        return f"Error in calculation: {str(e)}"




# Function to check if the query is student-related
def student_chatbot(student_input):
    try:
        # Check if the query is relevant to students
        if not is_student_query(student_input):
            return "This chatbot is only for educational questions. Please ask something related to your studies or school."

        # Detect if the query is a calculation
        if any(char.isdigit() for char in student_input) and any(op in student_input for op in ["+", "-", "*", "/", "^"]):
            return f"Calculator Result: {calculate_expression(student_input)}"

        # Detect if the query is a Wikipedia search
        if "wiki" in student_input.lower() or "information about" in student_input.lower():
            query = (
                student_input.replace("wiki", "")
                .replace("information about", "")
                .strip()
            )
            return f"Wikipedia Result: {search_wikipedia(query)}"

        # Detect if the query is a Taveli search
        if "search taveli" in student_input.lower():
            query = student_input.replace("search taveli", "").strip()
            return f"Taveli Result: {search_taveli(query)}"

        # Detect if spelling corrections are needed
        if "check spelling" in student_input.lower():
            query = student_input.replace("check spelling", "").strip()
            return f"Corrected Spelling: {check_spelling(query)}"

        # Default: Use the Groq model for student questions
        prompt = f"You are a helpful assistant for students. Answer the following question in a way that helps a student understand the topic better:\n{student_input}"
        response = llm.invoke(prompt)
        return response.content.strip()

    except Exception as e:
        return f"An error occurred: {str(e)}"


# Gradio Interface
interface = gr.Interface(
    fn=student_chatbot,
    inputs=gr.Textbox(label="Ask Your Question", placeholder="Enter your question here..."),
    outputs=gr.Textbox(label="Answer"),
    title="Student Helper Chatbot",
    description=(
        "Ask me any educational question, and I'll help you learn! Features include:\n"
        "- Math calculations (e.g., 2+2*3).\n"
        "- Wikipedia search (e.g., 'wiki [topic]').\n"
        "- Taveli search (e.g., 'search taveli [topic]').\n"
        "- Spell checking (e.g., 'check spelling [text]')."
    )
)

# Launch the interface
interface.launch()




Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2972bf0300fd725780.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
